# GPS: The General Problem Solver


In 1957 werkten Alan Newell en Herbert Simon aan een van de eerste AI programma's: de General Problem Solver. Het programma volgt de zogenaamde 'symbolic AI' aanpak. Dit betekent dat intelligentie geprogrameerd wordt door een probleem te representeren met symbolen binnen een programma en te manipuleren via logica en zoek algoritmen.

De verwachtingen van de symbolic AI aanpak waren in de begin jaren hoog. Zo hoog zelfs, dat men dacht dat intelligente computers binnen 10 jaar ontwikkeld zouden zijn. Dit was dus een verkeerde inschatting. Een van de problemen bij deze aanpak is dat het AI programma niet meer kennis heeft over het probleem dan dat er in de code geprogrammeerd is. 

Toch is de GPS een belangrijke mijlpaal geweest voor AI. Het was namelijk het eerste AI programma dat kennis over een probleem (de feiten database) en de strategie om het probleem op te lossen (de kennisregels) scheidde. 

Hieronder wordt beschreven hoe je de GPS kunt programmeren in Python. Hierbij wordt een vertaling gemaakt van LIPS naar Python uit het boek 'Paradigms of Artificial Intelligence Programming' van Peter Norvig.


## Means-End analyse

In hun boek "Human Problem Solving" uit 1972 beschrijven Newell en Simon het volgende scenario om te beschrijven hoe mensen problemen oplossen:

> Ik wil mijn zoon naar school brengen. Wat is het verschil met waar ik nu ben en waar ik heen wil? Dat is de afstand tussen mijn huis en de school. Hoe kan ik dit verschil veranderen? Dat kan met mijn auto. Mijn auto werkt echter niet. Wat is er voor nodig om mijn auto weer werkend te krijgen? Ik heb een nieuwe accu nodig. Hoe kom ik aan een nieuwe accu? Een accu kan ik bij de garage halen. Hoe weet de garage dat ik een accu nodig heb? Dan moet ik ze bellen. ... enzovoort

Deze aanpak waarbij je kijkt met welke aktie je het verschil tussen de huidige toestand en de gewenste toestand van een probleem kunt verkleinen, heet means-end analyse.


## Python implementatie van GPS


In [18]:
class Operators:
    """
    Acties die uitgevoerd kunnen worden om een (deel)probleem
    op te lossen.
    """
    def __init__(self):
        self.ops = []    # lijst met acties

    def make_op(self, action, preconds=[], add_list=[], del_list=[], front_of_list=False):
        """Voeg actie toe."""
        op = {'action'  : action, \
              'preconds': preconds, \
              'add_list': add_list, \
              'del_list': del_list }

        if front_of_list:
            self.ops.insert(0, op)
        else:
            self.ops.append(op)
        
    def find_ops(self, goal):
        """Zoek alle acties om een goal te bereiken."""
        return [op for op in self.ops if goal in op['add_list']]
    
    def apply(self, state, op):
        """Voer een actie uit."""
        return [s for s in state if s not in op['del_list']] + op['add_list']
        
    
ops = Operators()
    
ops.make_op(action   = 'drive-son-to-school', \
            preconds = ['son-at-home', 'car-works'], \
            add_list = ['son-at-school'], \
            del_list = ['son-at-home'])

ops.make_op(action   = 'shop-installs-battery', \
            preconds = ['car-needs-battery', 'shop-knows-problem', 'shop-has-money'], \
            add_list = ['car-works'], \
            del_list = [])

ops.make_op(action   = 'tell-shop-problem', \
            preconds = ['in-communication-with-shop'], \
            add_list = ['shop-knows-problem'])

ops.make_op(action   = 'telephone-shop', \
            preconds = ['know-phone-number'], \
            add_list = ['in-communication-with-shop'])

ops.make_op(action   = 'look-up-number', \
            preconds = ['have-phone-book'], \
            add_list = ['know-phone-number'])

ops.make_op(action   = 'give-shop-money', \
            preconds = ['have-money'], \
            add_list = ['shop-has-money'], \
            del_list = ['have-money'])


# The General Problem Solver
class GPS:
    def __init__(self, ops, state=[], goals=[]):
        self.state = state
        self.goals = goals
        self.ops = ops
        
    def achieve(self, goal):
        if goal in self.state:
            # Als goal al is bereikt in huidige state dan return True
            return True
        else:
            # Als goal nog niet is bereikt, zoek een actie waarmee goal
            # bereikt kan worden
            for op in self.ops.find_ops(goal):  
                # test of alle pre-conditities gerealiseerd kunnen worden
                if all(self.achieve(c) for c in op['preconds']):
                    print("EXECUTING {}".format(op['action']))
                    self.state = ops.apply(self.state, op)
                    return True
        return False
        
    def solve(self):
        # test of alle goals gerealiseerd kunnen worden
        if all(self.achieve(goal) for goal in self.goals):
            print("SOLVED")
        else:
            print("NOT SOLVED")
            
gps = GPS(ops)

In [19]:
gps.state = ['son-at-school']
gps.goals = ['son-at-school']
gps.solve()

SOLVED


In [20]:
gps.state = ['son-at-home', 'car-needs-battery', 'have-money', 'have-phone-book']
gps.goals = ['son-at-school']
gps.solve()

EXECUTING look-up-number
EXECUTING telephone-shop
EXECUTING tell-shop-problem
EXECUTING give-shop-money
EXECUTING shop-installs-battery
EXECUTING drive-son-to-school
SOLVED


In [21]:
gps.state = ['son-at-home', 'car-needs-battery', 'have-money']
gps.goals = ['son-at-school']
gps.solve()

NOT SOLVED


In [22]:
gps.state = ['son-at-home', 'car-works']
gps.goals = ['son-at-school']
gps.solve()

EXECUTING drive-son-to-school
SOLVED


## Beperkingen van GPS en hoe deze op te lossen

De General Problem Solver die hierboven beschreven staat heeft een aantal beperkingen. 

De eerste beperking heeft te maken met akties die geen impact hebben op de toestand van het probleem. De `add_list` en `del_list` zijn hierbij leeg. Een dergelijke actie uitvoeren heeft geen impact op de toestand. GPS moet hier rekening meehouden.

De tweede beperking heeft te maken met het bereiken van meerdere doelen op hetzelfde moment. Stel je wilt de doelen `have-money` en `son-at-school` bereiken. Als we dit probleem laten oplossen door GPS dan is het resultaat dat het probleem opgelost is.

In [23]:
gps.state = ['son-at-home', 'car-needs-battery', 'have-money', 'have-phone-book']
gps.goals = ['have-money', 'son-at-school']
gps.solve()

EXECUTING look-up-number
EXECUTING telephone-shop
EXECUTING tell-shop-problem
EXECUTING give-shop-money
EXECUTING shop-installs-battery
EXECUTING drive-son-to-school
SOLVED


In werkelijkheid klopt dit niet. Zodra de actie `give-shop-money` wordt uitgevoerd heb je geen geld meer. In de GPS functie zoals die hierboven staat wordt niet gecontroleerd of alle doelen op het zelfde moment bereikt zijn. In een aangepaste versie van GPS gebruiken we daarom een nieuwe functie `achieve_all` die hier rekening mee houdt.

De derde beperking van GPS zien we als we de doelen omgewisselen. Hoewel GPS wel het juiste resultaat geeft, worden er al akties uitgevoerd. Dit is ongewenst en wordt veroorzaakt dat planning en executie door elkaar heen lopen.

In [24]:
gps.state = ['son-at-home', 'car-needs-battery', 'have-money', 'have-phone-book']
gps.goals = ['son-at-school', 'have-money']
gps.solve()

EXECUTING look-up-number
EXECUTING telephone-shop
EXECUTING tell-shop-problem
EXECUTING give-shop-money
EXECUTING shop-installs-battery
EXECUTING drive-son-to-school
NOT SOLVED


De vierde beperking ontstaat door acties die lussen veroorzaken. Actie A heeft een preconditie die gerealiseerd kan worden door actie B, terwijl actie B een preconditie heeft die gerealiseerd kan worden door actie A.

In [25]:
gps.ops.make_op(action = 'ask-phone-number', \
                preconds = ['in-communication-with-shop'], \
                add_list = ['know-phone-number'], \
                front_of_list=True)
gps.state = ['son-at-home', 'car-needs-battery', 'have-money']
gps.goal = ['son-at-school']
gps.solve() # geeft fout!!

RecursionError: maximum recursion depth exceeded

Deze beperking kan opgelost worden door bij te houden aan welke doelen er gewerkt wordt. Zodra een lus wordt gedetecteerd moet de executie worden opgegeven.

De laatste beperking is dat GPS geen debug informatie laat zien. Als er geen oplossing is dan hebben we geen inzicht waarom dat zo is.

In [312]:
# Verbeterde General Problem Solver
class GPS2:
    def __init__(self, ops, state=[], goals=[]):
        self.state = state
        self.goals = goals
        self.ops = ops
        self.debug = False
        self.goal_stack = []
        self.plan = []
        
    def achieve_all(self, state, goals):
        """Realiseer elk doel en test dat ze gezamelijk bereikt worden."""
        for goal in goals:
            # debug
            if self.debug:
                print("{}GOAL: {}".format(len(self.goal_stack) * '  ', goal))
            
            self.goal_stack.insert(0, goal)
            result, state = self.achieve(state, goal)
            self.goal_stack.pop(0)
            
            if result==False:
                return False, state
            
        if all(g in state for g in goals):
            return True, state
        
        return False, state
        
    def achieve(self, state, goal):
        if goal in state:
            # Als goal al is bereikt in huidige state dan return True
            return True, state
        elif goal in self.goal_stack[1:]:
            # Als er al aan goal gewerkt wordt, return False om lussen te voorkomen
            return False, state

        # Als goal nog niet is bereikt, zoek een actie waarmee goal
        # bereikt kan worden
        for op in self.ops.find_ops(goal):  
            # debug
            if self.debug:
                print("{}CONSIDER {}".format((len(self.goal_stack)-1) * '  ', op['action']))

            # test of alle pre-conditities gerealiseerd kunnen worden
            result, state = self.achieve_all(state, op['preconds'])

            if result == True:
                # debug
                if self.debug:
                    print("{}Action {}".format((len(self.goal_stack)) * '  ', op['action']))
                self.plan.append(op['action'])
                return True, ops.apply(state, op)
        
        return False, state
            
    def solve(self, debug=False):
        self.debug = debug
                    
        # test of alle goals gerealiseerd kunnen worden
        self.goal_stack = []
        self.plan = []
        result, new_state = self.achieve_all(self.state, self.goals)
        if result:
            if len(self.plan) == 0:
                print("((START))")
            else:
                print("((START)")
                for a in self.plan[:-1]:
                    print(" (EXECUTING {})".format(a))
                print(" (EXECUTING {}))".format(self.plan[-1]))
            self.state = list(new_state)
        else:
            print("NOT SOLVED")
            
gps2 = GPS2(ops)

In [313]:
gps2.state = ['son-at-home', 'car-needs-battery', 'have-money', 'have-phone-book']
gps2.goals = ['son-at-school']
gps2.solve(debug=False)

((START)
 (EXECUTING look-up-number)
 (EXECUTING telephone-shop)
 (EXECUTING tell-shop-problem)
 (EXECUTING give-shop-money)
 (EXECUTING shop-installs-battery)
 (EXECUTING drive-son-to-school))


In [314]:
gps2.state = ['son-at-home', 'car-works']
gps2.goals = ['son-at-school']
gps2.solve(debug=False)

((START)
 (EXECUTING drive-son-to-school))


In [315]:
gps2.state = ['son-at-home', 'car-needs-battery', 'have-money', 'have-phone-book']
gps2.goals = ['have-money', 'son-at-school']
gps2.solve(debug=False)

NOT SOLVED


In [316]:
gps2.state = ['son-at-home', 'car-needs-battery', 'have-money', 'have-phone-book']
gps2.goals = ['son-at-school', 'have-money']
gps2.solve()

NOT SOLVED


In [317]:
gps2.state = ['son-at-home', 'car-needs-battery', 'have-money']
gps2.goals = ['son-at-school']
gps2.solve()

NOT SOLVED


In [318]:
gps2.state = ['son-at-home']
gps2.goals = ['son-at-home']
gps2.solve()

((START))


## Apen en bananen probleem

Om te laten zien dat GPS2 voor een willekeurig probleem domein gebruikt kan worden gaan we een andere probleem oplossen. Dit is een klassiek AI probleem met een aap en een banaan. Het probleem is als volgt:

Een aap staat in een kamer. In het midden van de kamer hangt een banaan aan het plafond met een touw. De aap kan hier niet direct bij. Er staat in de kamer een stoel die, als deze onder de banaan geplaatst wordt, de aap in staat stelt de banaan te pakken. Om het probleem iets moeilijker te maken geven we de aap een stuk speelgoed om vast te houden. Hij kan pas de banaan pakken als zijn handen leeg zijn.

Om dit probleem met GPS2 op te lossen hoeven we de code van GPS2 niet meer aan te passen. Wel moeten we nieuwe domein kennis (de acties, state en goal) invoeren.

In [319]:
banana_ops = Operators()
    
banana_ops.make_op(action   = 'climb-on-chair', \
                   preconds = ['chair-at-middle-room', 'at-middle-room', 'on-floor'], \
                   add_list = ['at-bananas', 'on-chair'], \
                   del_list = ['at-middle-room', 'on-floor'])

banana_ops.make_op(action   = 'push-chair-from-door-to-middle-room', \
                   preconds = ['chair-at-door', 'at-door'], \
                   add_list = ['chair-at-middle-room', 'at-middle-room'], \
                   del_list = ['chair-at-door', 'at-door'])

banana_ops.make_op(action   = 'walk-from-door-to-middle-room', \
                   preconds = ['at-door', 'on-floor'], \
                   add_list = ['at-middle-room'], \
                   del_list = ['at-door'])

banana_ops.make_op(action   = 'grasp-bananas', \
                   preconds = ['at-bananas', 'empty-handed'], \
                   add_list = ['has-bananas'], \
                   del_list = ['empty-handed'])

banana_ops.make_op(action   = 'drop-ball', \
                   preconds = ['has-ball'], \
                   add_list = ['empty-handed'], \
                   del_list = ['has-ball'])

banana_ops.make_op(action   = 'eat-bananas', \
                   preconds = ['has-bananas'], \
                   add_list = ['empty-handed', 'not-hungry'], \
                   del_list = ['has-bananas', 'hungry'])

gps2 = GPS2(banana_ops)

In [320]:
gps2.state = ['at-door', 'on-floor', 'has-ball', 'hungry', 'chair-at-door']
gps2.goals = ['not-hungry']
gps2.solve()

((START)
 (EXECUTING push-chair-from-door-to-middle-room)
 (EXECUTING climb-on-chair)
 (EXECUTING drop-ball)
 (EXECUTING grasp-bananas)
 (EXECUTING eat-bananas))


## Doolhof probleem

Een ander bekend AI probleem is het doolhof probleem. We moeten vanaf een start positie naar een eind positie lopen door een doolhof. GPS2 kan ook dit probleem oplossen. Wederom hoeft de code van GPS2 niet aangepast te worden. Wel moeten we weer domein kennis (acties, state en goal) definiëren.

```
---------------------
| 1   2   3   4 | 5 |
|-----------    |   |
| 6 | 7   8   9 | 10|
|        ----   |   |
| 11  12  13| 14| 15|
|    -----------    |
| 16  17| 18| 19  20|
|---                |
| 21  22  23  24| 25|
---------------------

```


In [321]:
maze_ops = Operators()

def make_maze_op(maze_ops, move_from, move_to):
    """Doolhof verplaats actie."""
    maze_ops.make_op(action   = "move from {} to {}".format(move_from, move_to), \
                     preconds = ["at {}".format(move_from)], \
                     add_list = ["at {}".format(move_to)], \
                     del_list = ["at {}".format(move_from)])

def make_maze_ops(maze_ops, A, B):
    """Maak doolhof verplaats actie in beide richtingen."""
    make_maze_op(maze_ops, A, B)
    make_maze_op(maze_ops, B, A)
    
    
make_maze_ops(maze_ops, 1, 2)
make_maze_ops(maze_ops, 2, 3)
make_maze_ops(maze_ops, 3, 4)
make_maze_ops(maze_ops, 4, 9)
make_maze_ops(maze_ops, 9, 14)
make_maze_ops(maze_ops, 9, 8)
make_maze_ops(maze_ops, 8, 7)
make_maze_ops(maze_ops, 7, 12)
make_maze_ops(maze_ops, 12, 13)
make_maze_ops(maze_ops, 12, 11)
make_maze_ops(maze_ops, 11, 6)
make_maze_ops(maze_ops, 11, 16)
make_maze_ops(maze_ops, 16, 17)
make_maze_ops(maze_ops, 17, 22)
make_maze_ops(maze_ops, 22, 21)
make_maze_ops(maze_ops, 22, 23)
make_maze_ops(maze_ops, 23, 18)
make_maze_ops(maze_ops, 23, 24)
make_maze_ops(maze_ops, 24, 19)
make_maze_ops(maze_ops, 19, 20)
make_maze_ops(maze_ops, 20, 15)
make_maze_ops(maze_ops, 15, 10)
make_maze_ops(maze_ops, 10, 5)
make_maze_ops(maze_ops, 20, 25)

gps2 = GPS2(maze_ops)

In [322]:
gps2.state = ['at 1']
gps2.goals = ['at 25']
gps2.solve()

((START)
 (EXECUTING move from 1 to 2)
 (EXECUTING move from 2 to 3)
 (EXECUTING move from 3 to 4)
 (EXECUTING move from 4 to 9)
 (EXECUTING move from 9 to 8)
 (EXECUTING move from 8 to 7)
 (EXECUTING move from 7 to 12)
 (EXECUTING move from 12 to 11)
 (EXECUTING move from 11 to 16)
 (EXECUTING move from 16 to 17)
 (EXECUTING move from 17 to 22)
 (EXECUTING move from 22 to 23)
 (EXECUTING move from 23 to 24)
 (EXECUTING move from 24 to 19)
 (EXECUTING move from 19 to 20)
 (EXECUTING move from 20 to 25))


## Block world


In [323]:
import itertools

block2_ops = Operators()

def move_op(block_ops, a, b, c):
    """Verplaats block a van b naar c."""
    block_ops.make_op(action   = "move {} from {} to {}".format(a, b, c), \
                      preconds = ["space-on-{}".format(a), "space-on-{}".format(c), "{}-on-{}".format(a, b)], \
                      add_list = move_ons(a, b, c), \
                      del_list = move_ons(a, c, b),
                      front_of_list = True)
    
def move_ons(a, b, c):
    if b=='table':
        return ["{}-on-{}".format(a, c)]
    return ["{}-on-{}".format(a, c), "space-on-{}".format(b),]
        
    
def make_block_ops(block_ops, blocks):
    for a in blocks:
        for b in blocks:
            if a != b:
                for c in blocks:
                    if (c != a) and (c != b):
                        move_op(block_ops, a, b, c)
                move_op(block_ops, a, 'table', b)
                move_op(block_ops, a, b, 'table')
        
make_block_ops(block2_ops, ['a', 'b'])
gps2 = GPS2(block2_ops)

In [324]:
block2_ops.ops

[{'action': 'move b from a to table',
  'add_list': ['b-on-table', 'space-on-a'],
  'del_list': ['b-on-a'],
  'preconds': ['space-on-b', 'space-on-table', 'b-on-a']},
 {'action': 'move b from table to a',
  'add_list': ['b-on-a'],
  'del_list': ['b-on-table', 'space-on-a'],
  'preconds': ['space-on-b', 'space-on-a', 'b-on-table']},
 {'action': 'move a from b to table',
  'add_list': ['a-on-table', 'space-on-b'],
  'del_list': ['a-on-b'],
  'preconds': ['space-on-a', 'space-on-table', 'a-on-b']},
 {'action': 'move a from table to b',
  'add_list': ['a-on-b'],
  'del_list': ['a-on-table', 'space-on-b'],
  'preconds': ['space-on-a', 'space-on-b', 'a-on-table']}]

In [325]:
gps2.state = ['a-on-table', 'b-on-table', 'space-on-a', 'space-on-b', 'space-on-table']
gps2.goals = ['a-on-b', 'b-on-table']
gps2.solve()

((START)
 (EXECUTING move a from table to b))


Wissel de blokken om.

In [326]:
gps2.state = ['a-on-b', 'b-on-table', 'space-on-a', 'space-on-table']
gps2.goals = ['b-on-a', 'a-on-table']
gps2.solve(debug=False)

((START)
 (EXECUTING move a from b to table)
 (EXECUTING move b from table to a))


In [594]:
block3_ops = Operators()
    
make_block_ops(block3_ops, ['a', 'b', 'c'])
gps2 = GPS2(block3_ops)

In [595]:
gps2.ops.ops

[{'action': 'move c from b to table',
  'add_list': ['c-on-table', 'space-on-b'],
  'del_list': ['c-on-b'],
  'preconds': ['space-on-c', 'space-on-table', 'c-on-b']},
 {'action': 'move c from table to b',
  'add_list': ['c-on-b'],
  'del_list': ['c-on-table', 'space-on-b'],
  'preconds': ['space-on-c', 'space-on-b', 'c-on-table']},
 {'action': 'move c from b to a',
  'add_list': ['c-on-a', 'space-on-b'],
  'del_list': ['c-on-b', 'space-on-a'],
  'preconds': ['space-on-c', 'space-on-a', 'c-on-b']},
 {'action': 'move c from a to table',
  'add_list': ['c-on-table', 'space-on-a'],
  'del_list': ['c-on-a'],
  'preconds': ['space-on-c', 'space-on-table', 'c-on-a']},
 {'action': 'move c from table to a',
  'add_list': ['c-on-a'],
  'del_list': ['c-on-table', 'space-on-a'],
  'preconds': ['space-on-c', 'space-on-a', 'c-on-table']},
 {'action': 'move c from a to b',
  'add_list': ['c-on-b', 'space-on-a'],
  'del_list': ['c-on-a', 'space-on-b'],
  'preconds': ['space-on-c', 'space-on-b', 'c-on-

In [329]:
gps2.state = ['a-on-b', 'b-on-c', 'c-on-table', 'space-on-a', 'space-on-table']
gps2.goals = ['b-on-a', 'c-on-b']
gps2.solve(debug=False)


((START)
 (EXECUTING move a from b to table)
 (EXECUTING move b from c to a)
 (EXECUTING move c from table to b))


In [330]:
gps2.state = ['a-on-b', 'b-on-c', 'c-on-table', 'space-on-a', 'space-on-table']
gps2.goals = ['c-on-b', 'b-on-a']
gps2.solve(debug=False)

NOT SOLVED


## Cake


In [331]:
cake_ops = Operators()
    
cake_ops.make_op(action   = 'eat-cake', \
                   preconds = ['see-cake', 'hungry'], \
                   add_list = ['not-hungry'], \
                   del_list = ['see-cake', 'hungry'])

cake_ops.make_op(action   = 'take-picture-cake', \
                   preconds = ['see-cake'], \
                   add_list = ['have-picture-cake'], \
                   del_list = [])

gps2 = GPS2(cake_ops)

In [332]:
gps2.state = ['hungry', 'see-cake']
gps2.goals = ['not-hungry', 'have-picture-cake']
gps2.solve()

NOT SOLVED


In [593]:
# Verbeterde General Problem Solver
# Deze vesie is niet gevoelig voor de volgorde
# van de doelen
class GPS3:
    def __init__(self, ops, state=[], goals=[]):
        self.state = state
        self.goals = goals
        self.ops = ops
        self.debug = False
        self.goal_stack = []
        self.plan = []
        
        self.levels = []
            
    def achieve_any_combination(self, state, goals, plan=[]):
        prev_plan = list(plan)
        prev_state = list(state)
        result = False
        for goals_perm in itertools.permutations(goals):
            result, state, plan = self.achieve_all(prev_state, goals_perm, prev_plan)
            if result:
                return True, state, plan
        return False, prev_state, prev_plan

    def achieve_all(self, state, goals, plan):
        """Realiseer elk doel en test dat ze gezamelijk bereikt worden."""
        prev_plan = list(plan)
        prev_state = list(state)
        result = False
        for goal in goals:
            # debug
            if self.debug:
                print("{}GOAL: {}".format(len(self.goal_stack) * '  ', goal))
            result, state, plan = self.achieve(state, goal, plan)
            if result==False:
                break
        if result:
            if all(g in state for g in goals):
                return True, state, plan
        return False, prev_state, prev_plan
        
    def achieve(self, state, goal, plan):
        if goal in state:
            # Als goal al is bereikt in huidige state dan return True
            return True, state, plan
        elif goal in self.goal_stack[1:]:
            # Als er al aan goal gewerkt wordt, return False om lussen te voorkomen
            return False, state, plan

        # Als goal nog niet is bereikt, zoek een actie waarmee goal
        # bereikt kan worden
        prev_state = list(state)
        prev_plan = list(plan)
        self.goal_stack.insert(0, goal)
        for op in self.ops.find_ops(goal):  
            # debug
            if self.debug:
                print("{}CONSIDER {}".format((len(self.goal_stack)-1) * '  ', op['action']))
            result = False
            for goal_perm in itertools.permutations(op['preconds']):
                state = prev_state
                plan = prev_plan
                for g in goal_perm:
                    result, state, plan = self.achieve(state, g, plan)
                    if result==False:
                        break
                if result:
                    break
            if result:
                # debug
                if self.debug:
                    print("{}Action {}".format((len(self.goal_stack)) * '  ', op['action']))
                plan.append(op['action'])
                self.goal_stack.pop(0)
                return True, self.ops.apply(state, op), plan

        self.goal_stack.pop(0)
        return False, prev_state, prev_plan
                
    def solve(self, debug=False):
        self.debug = debug
        
        # test of alle goals gerealiseerd kunnen worden
        self.goal_stack = []
        self.plan = []
        result, new_state, self.plan = self.achieve_any_combination(self.state, self.goals)

        if result:
            if len(self.plan) == 0:
                print("((START))")
            else:
                print("((START)")
                for a in self.plan[:-1]:
                    print(" (EXECUTING {})".format(a))
                print(" (EXECUTING {}))".format(self.plan[-1]))
            self.state = list(new_state)
        else:
            print("NOT SOLVED")
            
gps3 = GPS3(cake_ops)

In [596]:
gps3.state = ['hungry', 'see-cake']
gps3.goals = ['not-hungry', 'have-picture-cake']
gps3.solve()

((START)
 (EXECUTING take-picture-cake)
 (EXECUTING eat-cake))


In [597]:
gps3 = GPS3(block3_ops)
gps3.state = ['a-on-b', 'b-on-c', 'c-on-table', 'space-on-a', 'space-on-table']
gps3.goals = ['c-on-b', 'b-on-a']
gps3.solve(debug=False)

((START)
 (EXECUTING move a from b to table)
 (EXECUTING move b from c to a)
 (EXECUTING move c from table to b))


In [615]:
block4_ops = Operators()    
make_block_ops(block4_ops, ['a', 'b', 'c', 'd'])

In [616]:
gps = GPS3(block4_ops)
gps.state = ['a-on-b', 'b-on-c', 'c-on-d', 'd-on-table', 'space-on-a', 'space-on-table']
gps.goals = ['a-on-table', 'd-on-a']
gps.solve(debug=False)

((START)
 (EXECUTING move a from b to table)
 (EXECUTING move b from c to table)
 (EXECUTING move c from d to table)
 (EXECUTING move a from table to d)
 (EXECUTING move a from d to table)
 (EXECUTING move d from table to c)
 (EXECUTING move d from c to a))


In [620]:
block3_ops = Operators()    
make_block_ops(block3_ops, ['a', 'b', 'c'])
gps = GPS3(block3_ops)
gps.state = ['c-on-a', 'a-on-table', 'b-on-table', 'space-on-c', 'space-on-b', 'space-on-table']
gps.goals = ['c-on-table', 'a-on-b']
gps.solve(debug=False)

((START)
 (EXECUTING move c from a to b)
 (EXECUTING move c from b to table)
 (EXECUTING move a from table to c)
 (EXECUTING move a from c to b))


# Efficientie

GPS3 is niet efficient met het plannen van de acties. Om de minst mogelijke acties te plannen zouden we alle mogelijke plannen moeten bekijken en vergelijken met elkaar. Het plan met het minst aantal acties zou de voorkeur hebben. Dit vereist een andere "zoek strategie". Dit is een heel eigen onderzoek binnen de AI (waar we op deze website nog aandacht aan gaan besteden). Voorlopig gaan we GPR efficienter maken door de mogelijke acties die beschikbaar zijn om een doel te realiseren te ordenen naar het aantal precondities waaraan reeds wordt voldaan.

In [614]:
# Verbeterde General Problem Solver
# Deze versie selecteert de mogelijke acties
# op een efficientere manier.
class Operators:
    """
    Acties die uitgevoerd kunnen worden om een (deel)probleem
    op te lossen.
    """
    def __init__(self):
        self.ops = []    # lijst met acties

    def make_op(self, action, preconds=[], add_list=[], del_list=[], front_of_list=False):
        """Voeg actie toe."""
        op = {'action'  : action, \
              'preconds': preconds, \
              'add_list': add_list, \
              'del_list': del_list }

        if front_of_list:
            self.ops.insert(0, op)
        else:
            self.ops.append(op)
        
    def find_ops(self, goal, state=[]):
        """Zoek alle acties om een goal te bereiken."""
        ops = [op for op in self.ops if goal in op['add_list']]
        
        return sorted(ops, key = lambda x : len(x['preconds']) - \
                           len(set(state).intersection(x['preconds'])))
    
    def apply(self, state, op):
        """Voer een actie uit."""
        return [s for s in state if s not in op['del_list']] + op['add_list']

class GPS4:
    def __init__(self, ops, state=[], goals=[]):
        self.state = state
        self.goals = goals
        self.ops = ops
        self.debug = False
        self.goal_stack = []
        self.plan = []
        
        self.levels = []
            
    def achieve_any_combination(self, state, goals, plan=[]):
        prev_plan = list(plan)
        prev_state = list(state)
        result = False
        for goals_perm in itertools.permutations(goals):
            result, state, plan = self.achieve_all(prev_state, goals_perm, prev_plan)
            if result:
                return True, state, plan
        return False, prev_state, prev_plan

    def achieve_all(self, state, goals, plan):
        """Realiseer elk doel en test dat ze gezamelijk bereikt worden."""
        prev_plan = list(plan)
        prev_state = list(state)
        result = False
        for goal in goals:
            # debug
            if self.debug:
                print("{}GOAL: {}".format(len(self.goal_stack) * '  ', goal))
            result, state, plan = self.achieve(state, goal, plan)
            if result==False:
                break
        if result:
            if all(g in state for g in goals):
                return True, state, plan
        return False, prev_state, prev_plan
        
    def achieve(self, state, goal, plan):
        if goal in state:
            # Als goal al is bereikt in huidige state dan return True
            return True, state, plan
        elif goal in self.goal_stack[1:]:
            # Als er al aan goal gewerkt wordt, return False om lussen te voorkomen
            return False, state, plan

        # Als goal nog niet is bereikt, zoek een actie waarmee goal
        # bereikt kan worden
        prev_state = list(state)
        prev_plan = list(plan)
        self.goal_stack.insert(0, goal)
        for op in self.ops.find_ops(goal, prev_state):  
            # debug
            if self.debug:
                print("{}CONSIDER {}".format((len(self.goal_stack)-1) * '  ', op['action']))
            result = False
            for goal_perm in itertools.permutations(op['preconds']):
                state = prev_state
                plan = prev_plan
                for g in goal_perm:
                    result, state, plan = self.achieve(state, g, plan)
                    if result==False:
                        break
                if result:
                    break
            if result:
                # debug
                if self.debug:
                    print("{}Action {}".format((len(self.goal_stack)) * '  ', op['action']))
                plan.append(op['action'])
                self.goal_stack.pop(0)
                return True, self.ops.apply(state, op), plan

        self.goal_stack.pop(0)
        return False, prev_state, prev_plan
                
    def solve(self, debug=False):
        self.debug = debug
        
        # test of alle goals gerealiseerd kunnen worden
        self.goal_stack = []
        self.plan = []
        result, new_state, self.plan = self.achieve_any_combination(self.state, self.goals)

        if result:
            if len(self.plan) == 0:
                print("((START))")
            else:
                print("((START)")
                for a in self.plan[:-1]:
                    print(" (EXECUTING {})".format(a))
                print(" (EXECUTING {}))".format(self.plan[-1]))
            self.state = list(new_state)
        else:
            print("NOT SOLVED")
            

In [619]:
block3_ops = Operators()    
make_block_ops(block3_ops, ['a', 'b', 'c'])
gps = GPS4(block3_ops)
gps.state = ['c-on-a', 'a-on-table', 'b-on-table', 'space-on-c', 'space-on-b', 'space-on-table']
gps.goals = ['c-on-table', 'a-on-b']
gps.solve(debug=False)

((START)
 (EXECUTING move c from a to table)
 (EXECUTING move a from table to b))


Er blijven problemen die GPS toch nog niet kan oplossen. Zoals het ogenschijnlijke makkelijke probleem hieronder. Dit wordt de "Sussman anomaly" genoemd.

In [625]:
gps.state = ['c-on-a', 'a-on-table', 'b-on-table', 'space-on-c', 'space-on-b', 'space-on-table']
gps.goals = ['b-on-c', 'a-on-b']
gps.solve(debug=True)

GOAL: b-on-c
CONSIDER move b from table to c
  Action move b from table to c
GOAL: a-on-b
CONSIDER move a from c to b
  CONSIDER move c from a to table
    CONSIDER move b from c to table
      Action move b from c to table
    Action move c from a to table
  CONSIDER move a from table to c
    Action move a from table to c
  Action move a from c to b
GOAL: a-on-b
CONSIDER move a from c to b
  CONSIDER move c from a to table
    Action move c from a to table
  CONSIDER move a from table to c
    Action move a from table to c
  Action move a from c to b
GOAL: b-on-c
CONSIDER move b from table to c
  CONSIDER move c from b to table
    CONSIDER move c from table to b
    CONSIDER move c from a to b
      CONSIDER move c from b to a
      CONSIDER move c from table to a
        Action move c from table to a
      CONSIDER move c from b to a
      CONSIDER move c from table to a
        Action move c from table to a
      CONSIDER move c from b to a
      CONSIDER move c from table to a
  

In [ ]:
sorted()